In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('../input/VideoStarted.csv')

In [4]:
def make_timestamp(row):
    date_as_string = str(row['Date'])
    hour = str(row['Minute_Of_Day'] // (60))
    minute = str(row['Minute_Of_Day'] % 60)
    second = str(row['Second'])
    
    if len(hour) == 1:
        hour = '0' + hour
        
    if len(minute) == 1:
        minute = '0' + minute
    
    if len(second) == 1:
        second = '0' + second
    
    date_string = date_as_string[:4] + '-' + date_as_string[4:6] + '-' + date_as_string[6:] + ' ' + hour + ':' + minute + ':' + second
    datetime_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    return datetime_object

In [5]:
df['TimeStamp'] = df.apply(make_timestamp, axis=1)

In [20]:
def change_timestamp_to_epoch(row):
    pattern = '%Y-%m-%d %H:%M:%S'
    timestamp = str(row['TimeStamp'])
    epoch = int(time.mktime(time.strptime(timestamp, pattern)))
    return epoch

df['epoch'] = df.apply(change_timestamp_to_epoch,axis=1)

In [21]:
scaler = MinMaxScaler()
df['epoch'] = scaler.fit_transform(df[['epoch']])
df.head()

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,UserId,Date,Minute_Of_Day,Second,State,Genre,Category,ProgramType,Country,Device,OS,VideoId,TimeStamp,epoch
0,0280dfdd112732a3ac12b12dc770b7af,20170106,0,8,35,Romance,vod,Movies,1,1,2,36a27b379622f342ec87f9aafadb8f94,2017-01-06 00:00:08,0.000000
1,435d41ae019cb8db785483793859c9a8,20170106,0,28,35,Anime,vod,TV Shows,1,1,1,a4d9b88c7ed63d723c70b358a857719c,2017-01-06 00:00:28,0.000001
2,1faf0ce0b98e02e1568702f516f01a78,20170106,0,36,35,Drama,vod,TV Shows,1,1,1,806660cb47633263a24bbc53238a9a53,2017-01-06 00:00:36,0.000002
3,3ec691b9d2b5d53ef965fe59b1900b30,20170106,0,48,35,Drama,vod,TV Shows,1,2,2,385114825a85d6878e7a0978f9ba5546,2017-01-06 00:00:48,0.000003
4,ff230d487a7139b65f33b54a4cbd2d9e,20170106,0,56,35,Comedy,vod,Movies,1,1,1,368a480ec0ae105aee8320dd93483e39,2017-01-06 00:00:56,0.000004


In [22]:
df['epoch'].describe()

count    1.796525e+06
mean     6.956497e-01
std      2.527256e-01
min      0.000000e+00
25%      5.949850e-01
50%      7.617330e-01
75%      8.889386e-01
max      1.000000e+00
Name: epoch, dtype: float64

In [9]:
len(df['UserId'].unique())

165093

In [10]:
len(df['VideoId'].unique())

3550

In [11]:
x =  df[:100].groupby(['UserId', 'VideoId'])
x = x.first()
x

Date     ...           epoch
UserId                           VideoId                                        ...                
0280dfdd112732a3ac12b12dc770b7af 36a27b379622f342ec87f9aafadb8f94  20170106     ...      1483660808
04c1e14fd54a06c0d4fe98cf0cb8b04c 806660cb47633263a24bbc53238a9a53  20170106     ...      1483660932
057212548aff678c01fe2a55224cc2cc f3ecc88c6e53a990a75bd7e9b7ddb06e  20170106     ...      1483661264
079ec44549c738db2d55ff44b5b139f3 66ea27601d482d1857f6977450b2dc1c  20170106     ...      1483661876
087c741c167a352d07822fcab58046e5 2e47c28c5a2f4ac65315d0229447fdf8  20170106     ...      1483660908
0d164b3342b7c442b8a5cfc8a608bee1 afb85ac1bc0bb04bd7227ff5bf925bad  20170106     ...      1483660976
                                 d0c64e942ff427750850e49894ec48fe  20170106     ...      1483661120
0d2717ba754737c78430788ed77a1db7 675548e120ccb01387cc3ca9fed870a9  20170106     ...      1483660904
13d9f55e297d7d9c219fbe9085e1006c 8b6076b28243980a22f496bb127f3aec  20170106     ...      1483660884
16455c25d7a8af4084973ec08d320cd5 99e672aa585e8af17d01da8b839b6d18  20170106     ...      1483661416
19859f9b1e56c687b010391bc6670a76 57622239c0d0c605eb50e6923ebabb8a  20170106     ...      1483661508
1f9be9226d4393fff4b8d6b94e54c4dd 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106     ...      1483661248
                                 a4a773531e35757badfe759f6d69e7a7  20170106     ...      1483661196
                                 aa6dab080812d9e8b8e6b8f07c7a399d  20170106     ...      1483661156
1faf0ce0b98e02e1568702f516f01a78 806660cb47633263a24bbc53238a9a53  20170106     ...      1483660836
25e756b60e551802a441a50e794eff87 21f099b73c49eb58c49371abe5cfe63e  20170106     ...      1483661716
2956ff6e5c89beb4bd9e9590ba5596b4 67197f3b5830a66b117539268da239eb  20170106     ...      1483661712
32426aebb390a2561d793461a58b0ee8 1e16901cf325d640453b0d9279098803  20170106     ...      1483661096
                                 475774a85b02f70fbd08face24d6cec8  20170106     ...      1483661348
                                 e6ea7e407888ec088e91bafe475c91d0  20170106     ...      1483661144
32d8926c81149eb84ca5af5031a668a0 806660cb47633263a24bbc53238a9a53  20170106     ...      1483661824
335e967c2d1836de8e059d929c1a1eb3 58496b4becdf567f1b9f02943a6489a2  20170106     ...      1483661332
                                 993c5767292cd8922c422b348d582a5c  20170106     ...      1483661512
34a84047a6de70021be428b4ee3152bc e6ea7e407888ec088e91bafe475c91d0  20170106     ...      1483661316
3ec691b9d2b5d53ef965fe59b1900b30 385114825a85d6878e7a0978f9ba5546  20170106     ...      1483660848
42679123b0cc545eea930f1406704810 ef2f8cc077424405c8d09031ad76ef92  20170106     ...      1483661044
435d41ae019cb8db785483793859c9a8 a4d9b88c7ed63d723c70b358a857719c  20170106     ...      1483660828
4a48b6f152717235733e4bd48e5a3449 dad5c43748b229667804b2bcd8a40bf5  20170106     ...      1483661036
                                 e6ea7e407888ec088e91bafe475c91d0  20170106     ...      1483660956
4a9937fc586eddbdee50af81dec74364 66674d25b31f8b9e39389c3dcc94c9a0  20170106     ...      1483661828
...                                                                     ...     ...             ...
b0f1608f6f08dc67df769bd755fb1814 c0883669ac53bc724949ee0cf54457a9  20170106     ...      1483661108
                                 dfaf22b3245593f379be6579faa7535f  20170106     ...      1483661560
b2537964c1c6812227164f2d21e4f056 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106     ...      1483661836
                                 f88d90265ada11b808fc25417e78e3b5  20170106     ...      1483661884
b886b6f41c5e9667793644db95a6fe6a e6ea7e407888ec088e91bafe475c91d0  20170106     ...      1483661588
b8fd5e70eb720b258b6ad484d382d055 e6ea7e407888ec088e91bafe475c91d0  20170106     ...      1483661732
                                 ef2f8cc077424405c8d09031ad76ef92  20170106     ...      1483661744
ba53135c9063649e4a25138e81606198 82a7154f05d54bd1a29ec73b76289870  2017

In [23]:
user_video_mapper = dict()

def some_function(row):
    user_id = 'user_' + row['UserId']
    video_id = 'video_' + row['VideoId']
    epoch = row['epoch']
    
    if user_id not in user_video_mapper:
        user_video_mapper[user_id] = dict()
    
    video_info_of_user = user_video_mapper[user_id]
    
    if video_id not in video_info_of_user or video_info_of_user[video_id] < epoch:
        video_info_of_user[video_id] = epoch
    
    user_video_mapper[user_id] = video_info_of_user
    
df.apply(some_function, axis=1)
user_video_mapper

{'user_0280dfdd112732a3ac12b12dc770b7af': {'video_36a27b379622f342ec87f9aafadb8f94': 0.0},
 'user_435d41ae019cb8db785483793859c9a8': {'video_a4d9b88c7ed63d723c70b358a857719c': 1.4650739501576027e-06},
 'user_1faf0ce0b98e02e1568702f516f01a78': {'video_806660cb47633263a24bbc53238a9a53': 2.0511035359049856e-06,
  'video_e6ea7e407888ec088e91bafe475c91d0': 0.008466955403434895,
  'video_29145661d92ca9ba126436374184d043': 0.018640135935413582,
  'video_832a5de26fdcd68abd952a1d26af5b6a': 0.01914939564233009,
  'video_9e7eb053c618e8c13289c33dccf21a28': 0.031432868699766914,
  'video_8e52d2ec8e77c7df3b0aba1de3beedd9': 0.03144810546891108,
  'video_4b894a4f2ed090ead1ad053905ac5701': 0.05717275302339431,
  'video_74843ecae1130385a03df9bfa138af0f': 0.057602605721925215,
  'video_77ddf91bdbfa67161f98ad83d0161e69': 0.06519696307749712},
 'user_3ec691b9d2b5d53ef965fe59b1900b30': {'video_385114825a85d6878e7a0978f9ba5546': 2.9301479003152053e-06},
 'user_ff230d487a7139b65f33b54a4cbd2d9e': {'video_368a4

In [13]:
len(user_video_mapper.keys())

165093

In [ ]:
user_video_df = pd.DataFrame(user_video_mapper).T

In [15]:
user_video_df[:1][['video_36a27b379622f342ec87f9aafadb8f94']]

,video_36a27b379622f342ec87f9aafadb8f94
user_0280dfdd112732a3ac12b12dc770b7af,1.483661e+09


In [ ]:
user_video_df = user_video_df.replace(np.nan, 0)

In [ ]:
user_video_df.shape

In [ ]:
user_video_df.describe()

In [ ]:
user_video_cossine_similarity_df = cosine_similarity(user_video_df)